<a href="https://colab.research.google.com/github/Dahae6/Econ512_fall2024/blob/main/IO_HW1_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression
from statsmodels.sandbox.regression.gmm import IV2SLS
from scipy.optimize import fsolve

# Load the dataset
data = pd.read_csv('product_data.csv')

In [ ]:
# Problem 1(a)

# Number of consumers in each market
M_t = 100000

# Compute the share of the outside option (assuming it’s 1 minus the total share of soft drinks)
total_share = data.groupby('t')['market_share'].sum()
data = data.merge(total_share.rename('total_share'), on='t')
data['outside_share'] = 1 - data['total_share']

# Log of market share relative to the outside option's share
data['log_share'] = np.log(data['market_share']) - np.log(data['outside_share'])

# Define independent variables (price, sugar, caffeine, Diet, Regular)
# Create dummy variables based on the 'nest' variable

data['diet_d'] = np.where(data['nest'] == 'Diet', 1, 0)  # 1 if Diet, 0 otherwise
data['regular_d'] = np.where(data['nest'] == 'Regular', 1, 0)  # 1 if Regular, 0 otherwise
X = data[['price', 'sugar', 'caffeine', 'diet_d', 'regular_d']]

# The dependent variable is the log difference between market share and outside share
y = data['log_share']

# Perform OLS regression
ols_model = sm.OLS(y, X).fit()

# Print the summary of the OLS regression
print(ols_model.summary())


                            OLS Regression Results                            
Dep. Variable:              log_share   R-squared:                       0.885
Model:                            OLS   Adj. R-squared:                  0.884
Method:                 Least Squares   F-statistic:                     1912.
Date:                Sun, 22 Sep 2024   Prob (F-statistic):               0.00
Time:                        04:40:57   Log-Likelihood:                -1305.2
No. Observations:                1000   AIC:                             2620.
Df Residuals:                     995   BIC:                             2645.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
price         -1.6261      0.021    -77.352      0.0

In [ ]:
# Problem 1(b)

# Compute the share of the outside option (assuming it’s 1 minus the total share of soft drinks)
total_share = data.groupby('t')['market_share'].sum()
data = data.merge(total_share.rename('total_share'), on='t')
data['outside_share'] = 1 - data['total_share']

# Log of market share relative to the outside option's share
data['log_share'] = np.log(data['market_share']) - np.log(data['outside_share'])

# Create dummy variables for Diet and Regular
data['Diet'] = (data['nest'] == 'Diet').astype(int)
data['Regular'] = (data['nest'] == 'Regular').astype(int)


sugar = data['sugar']
caffeine = data['caffeine']
share = data['market_share']
diet = data['Diet']
regular = data['Regular']

# First Stage: Regress 'price' on instruments and exogenous variables

# Step 1: Create the instrument variables (taking into account the fact that prices affect in proportion to caffeine and sugar content)
data['instr_caffeine_price'] = data['caffeine_extract_price'] * data['caffeine']
data['instr_sugar_price'] = data['corn_syrup_price'] * data['sugar']

# Step 2: Combine the instruments
data['instr_price'] = data['instr_caffeine_price'] + data['instr_sugar_price']

# Exogenous variables (excluding price)
X_exog = data[['instr_price','caffeine','sugar','Diet', 'Regular']]

# Endogenous variable
y_endog = data['price']

# Perform the first stage OLS regression
first_stage_model = sm.OLS(y_endog, X_exog).fit()
data['predicted_price'] = first_stage_model.fittedvalues

# Instruments (created above)
instruments = data[['predicted_price', 'sugar', 'caffeine', 'Diet', 'Regular']]

# Second stage regression: Use predicted price instead of actual price
second_stage = sm.OLS(data['log_share'], instruments).fit()

# Step 5: Print the summary of the second stage results
print(second_stage.summary())

                            OLS Regression Results                            
Dep. Variable:              log_share   R-squared:                       0.200
Model:                            OLS   Adj. R-squared:                  0.196
Method:                 Least Squares   F-statistic:                     62.03
Date:                Sun, 22 Sep 2024   Prob (F-statistic):           7.93e-47
Time:                        04:41:06   Log-Likelihood:                -2274.7
No. Observations:                1000   AIC:                             4559.
Df Residuals:                     995   BIC:                             4584.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
predicted_price    -1.2109      0.406     

In [ ]:
# Assuming the estimated parameters from problem 1(b)
alpha = -1.2109
beta1 = 1.3348
beta2 = 1.3231
gamma_d = -2.6653
gamma_r = -6.5497

In [ ]:
#Problem 1 (c)

# Filter the data to include only Diet drinks
diet_data = data[data['Diet'] == 1]
regular_data = data[data['Regular'] == 1]


# Calculate the own-price derivatives and own-price elasticities
d_own_price_derivative = alpha * diet_data['market_share'] * (1 -diet_data['market_share'])
d_own_price_elasticity = d_own_price_derivative * diet_data['price'] / diet_data['market_share']

# d_mean_elasticity_regular = [regular]['own_price_elasticity'].mean()
d_mean_elasticity_diet = d_own_price_elasticity.mean()
print(f"Own-mean price elasticity for Diet drinks: {d_mean_elasticity_diet}")


# Calculate the own-price derivatives and own-price elasticities
r_own_price_derivative = alpha * regular_data['market_share'] * (1 - regular_data['market_share'])
r_own_price_elasticity = r_own_price_derivative * regular_data['price'] / regular_data['market_share']

r_mean_elasticity_regular = r_own_price_elasticity.mean()
print(f"Own-mean price elasticity for Regular drinks: {r_mean_elasticity_regular}")

Own-mean price elasticity for Diet drinks: -2.5538118020848186
Own-mean price elasticity for Regular drinks: -3.606697842999219


In [ ]:
# Problem 1(d)

product1_data = data[data['product_ID']== 1]
share_1 = product1_data['market_share']

# Cross-price derivative for all group

cross_price_derivatives = []
for t in data['t'].unique():
    for j in data[data['product_ID'] != 1]['product_ID'].unique():
        sjt = data[(data['t'] == t) & (data['product_ID'] == j)]['market_share'].values[0]
        cross_price_derivatives.append(-alpha * sjt * share_1)

# Calculate the cross-price elasticities for all j ≠ 1 and all t

cross_price_elasticities = []
for t in data['t'].unique():
    for j in data[data['product_ID'] != 1]['product_ID'].unique():
        sjt = data[(data['t'] == t) & (data['product_ID'] == j)]['market_share'].values[0]
        pj = data[(data['t'] == t) & (data['product_ID'] == j)]['price'].values[0]
        cross_price_elasticities.append(cross_price_derivatives.pop(0) * pj / sjt)

# Calculate the mean cross-price elasticity between product 1 and diet sodas, and between product 1 and regular sodas

mean_cross_price_elasticity_diet = np.mean([e for e, nest in zip(cross_price_elasticities, data[data['product_ID'] != 1]['nest']) if nest == 'Diet'])
mean_cross_price_elasticity_regular = np.mean([e for e, nest in zip(cross_price_elasticities, data[data['product_ID'] != 1]['nest']) if nest == 'Regular'])

print("Mean cross-price elasticity between product 1 and diet sodas:", mean_cross_price_elasticity_diet)
print("Mean cross-price elasticity between product 1 and regular sodas:", mean_cross_price_elasticity_regular)

Mean cross-price elasticity between product 1 and diet sodas: 0.19076487055996072
Mean cross-price elasticity between product 1 and regular sodas: 0.28434282928123555


In [ ]:
# Problem 1(e)


# Step 3: Define a function to generate the Jacobian matrix for a given time period
def jacobian_matrix(time_period, data, alpha):
    # Filter data for the given time period
    data_t = data[data['t'] == time_period]

    # Get the number of products in the time period
    n_products = len(data_t)

    # Initialize the Jacobian matrix
    jacobian = np.zeros((n_products, n_products))

    # Loop through products to compute the elements of the Jacobian
    for i in range(n_products):
        for j in range(n_products):
            s_j = data_t.iloc[j]['market_share']  # Market share of product j
            s_i = data_t.iloc[i]['market_share']  # Market share of product i
            if i == j:
                # Diagonal element: ∂s_j / ∂p_j = α * s_j * (1 - s_j)
                jacobian[i, j] = alpha * s_j * (1 - s_j)
            else:
                # Off-diagonal element: ∂s_j / ∂p_i = - α * s_j * s_i
                jacobian[i, j] = - alpha * s_j * s_i

    return jacobian

# Step 4: Generate the Jacobian matrix for the last time period (t = 100)
time_period = 100
jacobian = jacobian_matrix(time_period, data, alpha)

# Step 5: Display the Jacobian matrix
print(f"Jacobian matrix for time period {time_period}:")
print(jacobian)

Jacobian matrix for time period 100:
[[-7.14068846e-02  1.05873676e-04  7.78026153e-04  1.54836121e-03
   1.86582602e-02  2.97905551e-03  3.38853530e-02  7.33296374e-03
   3.10626249e-03  2.75347287e-03]
 [ 1.05873676e-04 -1.68005792e-03  1.71772671e-05  3.41847301e-05
   4.11937205e-04  6.57716094e-05  7.48121072e-04  1.61897227e-04
   6.85800859e-05  6.07911942e-05]
 [ 7.78026153e-04  1.71772671e-05 -1.22370669e-02  2.51210831e-04
   3.02717287e-03  4.83331026e-04  5.49766271e-03  1.18972234e-03
   5.03969472e-04  4.46731811e-04]
 [ 1.54836121e-03  3.41847301e-05  2.51210831e-04 -2.41044381e-02
   6.02442093e-03  9.61884130e-04  1.09409788e-02  2.36768379e-03
   1.00295700e-03  8.89047500e-04]
 [ 1.86582602e-02  4.11937205e-04  3.02717287e-03  6.02442093e-03
  -2.23894555e-01  1.15910191e-02  1.31842382e-01  2.85313660e-02
   1.20859607e-02  1.07133139e-02]
 [ 2.97905551e-03  6.57716094e-05  4.83331026e-04  9.61884130e-04
   1.15910191e-02 -4.54882857e-02  2.10505037e-02  4.55543669e

Q2. Estimation equation for nested logit model is as follows:
$log𝑠_{jt}−log𝑠_0 = \alpha p_{jt} + \beta_1 sug_{jt} + \beta_2 𝑐𝑎𝑓_{𝑗𝑡} + \gamma_d 𝐷𝑖𝑒𝑡_{𝑗} + \gamma_r 𝑅𝑒𝑔𝑢𝑙𝑎𝑟_{𝑗} + \sigma log 𝑠_{𝑗𝑡/𝑔} + \xi_{𝑗𝑡}$

In [ ]:
data['nest_total_share'] = data.groupby(['nest', 't'])['market_share'].transform('sum')

# Step 2: Calculate delta (log of market share divided by sum of market share in the nest)
data['s_jt_g'] = data['market_share'] / data['nest_total_share']
data['logs_jt_g'] = np.log(data['s_jt_g'])
data['s_g'] = data.groupby('nest')['market_share'].transform('sum')
data['s_jt'] = data['market_share']

We use identical instruments as question 1 (b).

In [ ]:
# Problem 2(a)

# Instruments (created above)
instruments = data[['predicted_price', 'sugar', 'caffeine', 'Diet', 'Regular', 'logs_jt_g']]

# Second stage regression: Use predicted price instead of actual price
nested_reg = sm.OLS(data['log_share'], instruments).fit()

# Step 5: Print the summary of the second stage results
print(nested_reg.summary())

                            OLS Regression Results                            
Dep. Variable:              log_share   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                 2.366e+04
Date:                Sun, 22 Sep 2024   Prob (F-statistic):               0.00
Time:                        04:41:23   Log-Likelihood:                 7.8483
No. Observations:                1000   AIC:                            -3.697
Df Residuals:                     994   BIC:                             25.75
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
predicted_price    -0.1840      0.042     

2 (b): $$
\frac{\partial s_{jt}}{\partial p_{jt}} = \frac{\alpha}{1-\sigma}s_{jt}(1-\sigma\bar{s}_{jt|g}-(1-\sigma)s_{jt})$$

$$
\frac{\partial s_{jt}}{\partial p_{jt}} \frac{p_{jt}}{s_{jt}} = \{\frac{\alpha}{1-\sigma}s_{jt}(1-\sigma\bar{s}_{jt|g}-(1-\sigma)s_{jt})\}\frac{p_{jt}}{s_{jt}}
$$

In [ ]:
# Estimated parameters from Problem 2(a)
alpha = -0.1840
sigma = 0.9797

In [ ]:
# Problem 2(b)

def calculate_price_derivatives_and_elasticities(data, alpha):
    # Own-price derivatives
    coef_der = alpha / (1-sigma)
    data['own_price_derivative'] = coef_der * data['s_jt'] * (1 - sigma * data['s_jt_g'] - (1 - sigma) * data['s_jt'])
    # Own-price elasticities
    data['own_price_elasticity'] = data['own_price_derivative'] * data['price'] / data['s_jt']

    return data

# Apply the derivative and elasticity calculation
data = calculate_price_derivatives_and_elasticities(data, alpha)

# Calculate the mean elasticities for regular and diet drinks
mean_elasticity_diet = data[data['Diet'] == 1]['own_price_elasticity'].mean()
mean_elasticity_regular = data[data['Regular'] == 1]['own_price_elasticity'].mean()

# Display the results
print(f'Mean own-price elasticity for Diet drinks: {mean_elasticity_diet}')
print(f'Mean own-price elasticity for Regular drinks: {mean_elasticity_regular}')

Mean own-price elasticity for Diet drinks: -16.300934428812113
Mean own-price elasticity for Regular drinks: -24.728992352331595


2 (c)


Case (1) the other product is in the same group:
$$
\frac{\partial s_{jt}}{\partial p_{kt}} = -\alpha s_{kt} (s_{jt} + \frac{\sigma}{1-\sigma}\bar{s}_{jt|g})
$$

$$
\frac{\partial s_{jt}}{\partial p_{kt}} \frac{p_{kt}}{s_{jt}} = -\alpha s_{kt} (s_{jt} + \frac{\sigma}{1-\sigma}\bar{s}_{jt|g}) \frac{p_{kt}}{s_{jt}}
$$

Case (2) the other product is in a different group:

$$
\frac{\partial s_{jt}}{\partial p_{kt}} = - \alpha s_{jt|g} s_g s_{jt|k} s_i = -\alpha s_{jt} s_{kt}
$$

$$
\frac{\partial s_{jt}}{\partial p_{kt}} \frac{p_{kt}}{s_{jt}} = -\alpha p_{kt} s_{kt}
$$

In [ ]:
# problem 2(c)

data['nest_total_share'] = data.groupby(['nest', 't'])['market_share'].transform('sum')
data['s_jt_g'] = data['market_share'] / data['nest_total_share']

# Get the utility, market share, and price for product 1
product1_data = data[data['product_ID'] == 1]
share_1 = product1_data['s_jt'].values[0]
price_1 = product1_data['price'].values[0]
s_1t_g = product1_data['s_jt_g'].values[0]

# Initialize lists to store cross-price derivatives and elasticities
cross_price_derivatives = []
cross_price_elasticities = []

# Calculate cross-price derivatives and elasticities for all products except product 1
for t in data['t'].unique():
    for j in data[data['product_ID'] != 1]['product_ID'].unique():
        sjt = data[(data['t'] == t) & (data['product_ID'] == j)]['market_share'].values[0]
        pj = data[(data['t'] == t) & (data['product_ID'] == j)]['price'].values[0]
        nest_j = data[(data['t'] == t) & (data['product_ID'] == j)]['nest'].values[0]
        s_jt_g = data[(data['t'] == t) & (data['product_ID'] == j)]['s_jt_g'].values[0]


        # If product j is in the same nest as product 1
        if nest_j == product1_data['nest'].values[0]:
            # Cross-price derivative (within-nest effect)
            coef1 = alpha * sigma / ( 1 - sigma)
            cross_price_derivative =  - coef1 * share_1 * s_jt_g - alpha * share_1 * sjt

        else:
            # Cross-price derivative (between-nest effect)
            cross_price_derivative = - alpha * sjt * share_1

        cross_price_derivatives.append(cross_price_derivative)

        # Calculate cross-price elasticity
        cross_price_elasticity = cross_price_derivative * (price_1 / sjt)
        cross_price_elasticities.append(cross_price_elasticity)

# Calculate mean cross-price elasticity between product 1 and diet sodas
mean_cross_price_elasticity_diet = np.mean([e for e, nest in zip(cross_price_elasticities, data[data['product_ID'] != 1]['nest']) if nest == 'Diet'])

# Calculate mean cross-price elasticity between product 1 and regular sodas
mean_cross_price_elasticity_regular = np.mean([e for e, nest in zip(cross_price_elasticities, data[data['product_ID'] != 1]['nest']) if nest == 'Regular'])

# Print results
print("Mean cross-price elasticity between product 1 and diet sodas:", mean_cross_price_elasticity_diet)
print("Mean cross-price elasticity between product 1 and regular sodas:", mean_cross_price_elasticity_regular)

Mean cross-price elasticity between product 1 and diet sodas: 8.498508506171504
Mean cross-price elasticity between product 1 and regular sodas: 0.05755371263073479


Problem 2 (d).

For Diet sodas, its demand reacts more sensitively to its own price changes both within nests and across nests compared to Problem 1. For Regular sodas, although its demand becomes more sensitive to its own price change, it becomes less sensitive to the Diet soda's price changes both within and across nests.

In [ ]:
# Problem 2(e)

# Define a function to generate the Jacobian matrix ∆(p)
def calculate_jacobian(data, alpha, sigma):
    n = len(data)  # Number of products
    jacobian_matrix = np.zeros((n, n))  # Initialize Jacobian matrix

    # Loop over all products to compute derivatives
    for i in range(n):
        for j in range(n):
            if i == j:  # Own-price derivative
                jacobian_matrix[i, j] = (alpha * data['s_jt'].iloc[i] * (1 - sigma * data['s_jt_g'].iloc[i] - (1 - sigma) * data['s_jt'].iloc[i])/ (1 - sigma))
            else:  # Cross-price derivative
                if data['nest'].iloc[i] == data['nest'].iloc[j]:  # If in the same nest
                    jacobian_matrix[i, j] = - data['s_jt'].iloc[j] * data['s_jt_g'].iloc[i] * alpha * sigma / (1-sigma) - alpha * data['s_jt'].iloc[i] * data['s_jt'].iloc[j]
                else:  # If in different nests
                    jacobian_matrix[i, j] = - (alpha * data['s_jt'].iloc[j] * data['s_jt'].iloc[i])

    return jacobian_matrix

# Apply the Jacobian calculation for a specific period (e.g., t = 100)
time_period = 100
data_t = data[data['t'] == time_period]  # Filter data for the selected period

# Calculate the Jacobian matrix for the selected period
jacobian_matrix = calculate_jacobian(data_t, alpha, sigma)

# Print the Jacobian matrix for the last time period (t = 100)
print(f"Jacobian matrix for time period {time_period}:\n", jacobian_matrix)

Jacobian matrix for time period 100:
 [[-4.66150751e-01  2.30167912e-03  1.69141813e-02  3.36611592e-02
   4.05628005e-01  4.52676699e-04  5.14898418e-03  1.11426652e-03
   4.72006192e-04  4.18398718e-04]
 [ 2.30167912e-03 -1.25425423e-02  3.73431419e-04  7.43171319e-04
   8.95545806e-03  9.99419947e-06  1.13679311e-04  2.46007844e-05
   1.04209562e-05  9.23740996e-06]
 [ 1.69141813e-02  3.73431419e-04 -8.97996885e-02  5.46128883e-03
   6.58103210e-02  7.34436442e-05  8.35386852e-04  1.80781989e-04
   7.65797199e-05  6.78822804e-05]
 [ 3.36611592e-02  7.43171319e-04  5.46128883e-03 -1.73304369e-01
   1.30970081e-01  1.46161268e-04  1.66251557e-03  3.59776875e-04
   1.52402419e-04  1.35093517e-04]
 [ 4.05628005e-01  8.95545806e-03  6.58103210e-02  1.30970081e-01
  -6.41112141e-01  1.76129121e-03  2.00338577e-02  4.33542930e-03
   1.83649911e-03  1.62792117e-03]
 [ 4.52676699e-04  9.99419947e-06  7.34436442e-05  1.46161268e-04
   1.76129121e-03 -3.33408205e-01  2.38200686e-01  5.15478471

In [ ]:
# problem 3(a)


# Step 1: Set the number of consumers in the market (M_t)
M_t = 100000

# Step 2: Calculate the quantity sold (q_jt = s_jt * M_t)
data['quantity_sold'] = data['market_share'] * M_t

# Step 3: Use the own-price derivatives to compute marginal cost
# The formula for marginal cost is: c_jt = p_jt + (s_jt / ∂s_jt / ∂p_jt)
data['marginal_cost'] = data['price'] + (data['market_share'] / data['own_price_derivative'])

# Step 4: Print the marginal costs
print(data[['product_ID', 'price', 'marginal_cost']])

# Step 5: Calculate the mean Lerner index for these products
# Lerner index: (p_jt - c_jt) / p_jt
data['lerner_index'] = (data['price'] - data['marginal_cost']) / data['price']
mean_lerner_index = data['lerner_index'].mean()

print("Mean Lerner Index for these products:", mean_lerner_index)


     product_ID     price  marginal_cost
0             1  2.814362       2.641139
1             2  2.935735       2.785182
2             3  2.467309       2.314064
3             4  1.543958       1.430979
4             5  1.495961       1.379585
..          ...       ...            ...
995           6  3.183553       3.066297
996           7  4.671217       4.334632
997           8  3.134615       3.005507
998           9  3.196304       3.078733
999          10  3.082792       2.966091

[1000 rows x 3 columns]
Mean Lerner Index for these products: 0.05740749337498979


Problem 3(b)

First, I will extract own price derivatives and cross price derivatives that will be used in 3 (b)

In [ ]:
# Define a function to generate the Jacobian matrix ∆(p)
def calculate_jacobian(data, alpha, sigma):
    n = len(data)  # Number of products
    jacobian_matrix = np.zeros((n, n))  # Initialize Jacobian matrix

    # Loop over all products to compute derivatives
    for i in range(n):
        for j in range(n):
            if i == j:  # Own-price derivative
                jacobian_matrix[i, j] = alpha * data['s_jt'].iloc[i] * (1 - sigma * data['s_jt_g'].iloc[i] - (1 - sigma) * data['s_jt'].iloc[i])/ (1 - sigma)
            else:  # Cross-price derivative
                if data['nest'].iloc[i] == data['nest'].iloc[j]:  # If in the same nest
                    jacobian_matrix[i, j] = - data['s_jt'].iloc[j] * data['s_jt_g'].iloc[i] * alpha * sigma / (1-sigma) - alpha * data['s_jt'].iloc[i] * data['s_jt'].iloc[j]
                else:  # If in different nests
                    jacobian_matrix[i, j] = - (alpha * data['s_jt'].iloc[j] * data['s_jt'].iloc[i])

    return jacobian_matrix

# Apply the Jacobian calculation for a specific period (e.g., t = 100)
time_period = 100
data_t = data[data['t'] == time_period]  # Filter data for the selected period

# Calculate the Jacobian matrix for the selected period
jacobian_matrix = calculate_jacobian(data_t, alpha, sigma)

# Extract own-price derivatives (diagonal elements)
own_price_derivatives = np.diag(jacobian_matrix)

# Extract cross-price derivatives (off-diagonal elements)
cross_price_derivatives = jacobian_matrix - np.diagflat(own_price_derivatives)

# Extract own and cross-price derivatives for products 1 and 2
# Assuming product IDs for 1 and 2 correspond to rows 0 and 1 in the data
own_price_derivative_1 = own_price_derivatives[0]
own_price_derivative_2 = own_price_derivatives[1]

cross_price_derivative_12 = jacobian_matrix[0, 1]  # Effect of p2 on s1
cross_price_derivative_21 = jacobian_matrix[1, 0]  # Effect of p1 on s2



Then, we solve the optimization problem as follows:


Objective Function:\
$\Pi_{ft} = (p_1 - c_1)M_t + (p_2 - c_2)M_t$


FOCs:

$ \frac{\partial \Pi_{ft}}{\partial p_{1t}} = s_{1t}M_t + (p_{1t} - c_{1t})\frac{\partial s_{1t}}{\partial p_{1t}}M_t + (p_{2t}-c_{2t})\frac{\partial s_{2t}}{\partial p_{1t}}M_t = 0$

$ \frac{\partial \Pi_{ft}}{\partial p_{2t}} = s_{2t}M_t + (p_{2t} - c_{2t})\frac{\partial s_{2t}}{\partial p_{2t}}M_t + (p_{1t}-c_{1t})\frac{\partial s_{1t}}{\partial p_{2t}}M_t = 0$


In [ ]:
# Filter data for t = 100 (the last time period)
data_t100 = data[data['t'] == 100]

# Market size (M_t = 100,000, given in the problem description)
M_t = 100000

# Extract necessary variables for products 1 and 2 at t = 100
market_share_1 = data_t100.loc[data_t100['product_ID'] == 1, 'market_share'].values[0]
market_share_2 = data_t100.loc[data_t100['product_ID'] == 2, 'market_share'].values[0]

# Extract marginal costs for products 1 and 2 at t = 100
c1 = data_t100.loc[data_t100['product_ID'] == 1, 'marginal_cost'].values[0]
c2 = data_t100.loc[data_t100['product_ID'] == 2, 'marginal_cost'].values[0]

# Extract prices for products 1 and 2 at t = 100
price_1 = data_t100.loc[data_t100['product_ID'] == 1, 'price'].values[0]
price_2 = data_t100.loc[data_t100['product_ID'] == 2, 'price'].values[0]
print(f"Initial price for product 1: {price_1}")
print(f"Initial price for product 2: {price_2}")

# We already extracted own-price and cross-price elasticities for products 1 and 2 at t = 100 from above

# Define the FOC system for two products (products 1 and 2) using actual data
def FOCs(variables):
    p1, p2 = variables
    s1 = market_share_1
    s2 = market_share_2

    # FOC for p1 (including cross-price effect of p2 on p1)
    FOC1 = s1*M_t + (p1 - c1) * own_price_derivative_1 * M_t + (p2 - c2) * cross_price_derivative_12 * M_t

    # FOC for p2 (including cross-price effect of p1 on p2)
    FOC2 = s2 * M_t + (p2 - c2) * own_price_derivative_2 * M_t + (p1 - c1) * cross_price_derivative_21 * M_t

    return [FOC1, FOC2]

# Initial guess for prices
initial_guess = [price_1, price_2]

# Solve the system of FOCs
solution = fsolve(FOCs, initial_guess)

# Output the solution
p1_optimal, p2_optimal = solution
print(f"Optimal price for product 1: {p1_optimal}")
print(f"Optimal price for product 2: {p2_optimal}")




Initial price for product 1: 3.179470747
Initial price for product 2: 1.907350248
Optimal price for product 1: 3.180140634102441
Optimal price for product 2: 1.9322469848657724


In [ ]:
# Problem 3(c)

# Load the dataset (assuming it's already loaded as 'data')

# Filter data for t = 100 (the last time period)
data_t100 = data[data['t'] == 100]

# Market size (M_t = 100,000)
M_t = 100000

# Extract prices for all products before solving for new optimal prices
prices_before = data_t100[['product_ID', 'price', 'Diet', 'Regular']]

# Solve for the optimal prices for the merging firms (products 1 and 2) using the existing code
# We assume that `FOCs` function and optimal prices for product 1 and 2 (`p1_optimal`, `p2_optimal`) are already computed

# Update prices for the merging firms (product 1 and 2)
data_t100.loc[data_t100['product_ID'] == 1, 'price'] = p1_optimal
data_t100.loc[data_t100['product_ID'] == 2, 'price'] = p2_optimal

# Extract prices after solving for optimal prices
prices_after = data_t100[['product_ID', 'price', 'Diet', 'Regular']]

# Calculate the average price for merging firms' products before and after the merger
merging_firm_products = [1, 2]
average_price_before_merger = prices_before[prices_before['product_ID'].isin(merging_firm_products)]['price'].mean()
average_price_after_merger = prices_after[prices_after['product_ID'].isin(merging_firm_products)]['price'].mean()

# Calculate the average price for competing products in the same nest (Diet or Regular)
# Products in the same nest as merging firms (filter by diet_dummy or regular_dummy)
diet_products = data_t100[data_t100['Diet'] == 1]
regular_products = data_t100[data_t100['Regular'] == 1]

competing_diet_prices_before = prices_before[(prices_before['Diet'] == 1) & (~prices_before['product_ID'].isin(merging_firm_products))]['price'].mean()
competing_regular_prices_before = prices_before[(prices_before['Regular'] == 1) & (~prices_before['product_ID'].isin(merging_firm_products))]['price'].mean()

competing_diet_prices_after = prices_after[(prices_after['Diet'] == 1) & (~prices_after['product_ID'].isin(merging_firm_products))]['price'].mean()
competing_regular_prices_after = prices_after[(prices_after['Regular'] == 1) & (~prices_after['product_ID'].isin(merging_firm_products))]['price'].mean()

# Print the results for Problem 3(c)
print(f"Average price of merging firms' products before the merger: {average_price_before_merger}")
print(f"Average price of merging firms' products after the merger: {average_price_after_merger}")
print(f"Average price of competing products in the same nest (Diet) before the merger: {competing_diet_prices_before}")
print(f"Average price of competing products in the same nest (Diet) after the merger: {competing_diet_prices_after}")
print(f"Average price of competing products in the same nest (Regular) before the merger: {competing_regular_prices_before}")
print(f"Average price of competing products in the same nest (Regular) after the merger: {competing_regular_prices_after}")



Average price of merging firms' products before the merger: 2.5434104975
Average price of merging firms' products after the merger: 2.556193809484107
Average price of competing products in the same nest (Diet) before the merger: 2.2272790473333335
Average price of competing products in the same nest (Diet) after the merger: 2.2272790473333335
Average price of competing products in the same nest (Regular) before the merger: 3.4536960544000004
Average price of competing products in the same nest (Regular) after the merger: 3.4536960544000004


In [ ]:
# Problem 3(d)


# Filter data for t = 100 (the last time period)
data_t100 = data[data['t'] == 100]

# Market size (M_t = 100,000, given in the problem description)
M_t = 100000

# Extract necessary variables (prices, marginal costs, market shares)
prices = data_t100['price'].values
marginal_costs = data_t100['marginal_cost'].values
market_shares = data_t100['market_share'].values

# Number of products
n = len(data_t100)

# Own-price and cross-price derivatives are already calculated via the Jacobian
jacobian_matrix = calculate_jacobian(data_t100, alpha, sigma)

# Define the joint profit function's FOCs for colluding firms
def collusion_FOCs(prices):
    FOCs = []

    # Loop through each product and define its FOC
    for i in range(n):
        # Initialize the cross-price derivative sum for product i
        cross_price_derivative_sum = 0

        # Loop over all products to compute own-price and cross-price derivatives
        for j in range(n):
            if i == j:
                # Own-price derivative for product i
                own_price_derivative = jacobian_matrix[i, i]
            else:
                # Cross-price derivative for product i with respect to price j
                cross_price_derivative_sum += (prices[j] - marginal_costs[j]) * jacobian_matrix[j, i]

        # FOC for product i: sum the own-price and cross-price effects
        FOC = market_shares[i] + (prices[i] - marginal_costs[i]) * own_price_derivative + cross_price_derivative_sum
        FOCs.append(FOC)

    return FOCs



# Initial guess for the prices (use the current prices)
initial_guess = prices

# Solve the system of FOCs for the colluding firms
optimal_prices_collusion = fsolve(collusion_FOCs, initial_guess)

# Update the dataset with the new collusion prices
data_t100['price_collusion'] = optimal_prices_collusion

# Calculate and print the average prices before and after collusion
average_price_before_collusion = data_t100['price'].mean()
average_price_after_collusion = data_t100['price_collusion'].mean()

print(f"Average price before collusion: {average_price_before_collusion}")
print(f"Average price after collusion: {average_price_after_collusion}")

# Print the optimal prices under collusion for each product
for idx, price in enumerate(optimal_prices_collusion):
    print(f"Optimal collusion price for product {data_t100['product_ID'].values[idx]}: {price}")


Average price before collusion: 2.9037138409
Average price after collusion: 1600.1660912535667
Optimal collusion price for product 1: 1600.474433482728
Optimal collusion price for product 2: 1599.2265398334923
Optimal collusion price for product 3: 1599.1384035650715
Optimal collusion price for product 4: 1599.8463432562132
Optimal collusion price for product 5: 1599.3741187157689
Optimal collusion price for product 6: 1600.4962597924327
Optimal collusion price for product 7: 1601.764594716376
Optimal collusion price for product 8: 1600.4354699851256
Optimal collusion price for product 9: 1600.5086955228437
Optimal collusion price for product 10: 1600.3960536656118


C:\Users\LG\AppData\Local\Temp\ipykernel_23920\1807599440.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_t100['price_collusion'] = optimal_prices_collusion
